In [2]:
# !pip3 install malaya -U --no-deps
# !pip3 install unidecode dateparser ftfy herpetologist networkx==2.5.1 sklearn sentencepiece
# !wget https://raw.githubusercontent.com/google-research-datasets/Taskmaster/master/TM-2-2020/data/movies.json

In [3]:
import malaya
import json
from tqdm import tqdm
import copy
import utils

2021-10-03 22:15:05.729584: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/.local/lib/python3.8/site-packages/malaya_boilerplate/frozen_graph.py:24: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(


In [4]:
transformer = malaya.translation.en_ms.transformer()

2021-10-03 22:15:10.075432: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-03 22:15:10.080067: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-03 22:15:10.149998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-03 22:15:10.151541: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 68 deviceMemorySize: 9.78GiB deviceMemoryBandwidth: 707.88GiB/s
2021-10-

In [5]:
with open('movies.json') as fopen:
    data = json.load(fopen)
len(data)

3056

In [6]:
texts = []
for row in tqdm(data):
    for u in row['utterances']:
        texts.append(u['text'])
        if 'segments' in u:
            for s in u['segments']:
                texts.append(s['text'])
texts = list(set(texts))
len(texts)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3056/3056 [00:00<00:00, 160632.02it/s]


50304

In [7]:
results = {}
batch_size = 10
for i in tqdm(range(0, len(texts), batch_size)):
    b = texts[i: i + batch_size]
    b_ = transformer.greedy_decoder(b)
    for k in range(len(b)):
        results[b[k]] = b_[k]

  0%|                                                                                                                                                             | 0/5031 [00:00<?, ?it/s]2021-10-03 22:15:14.053957: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2496000000 Hz
2021-10-03 22:15:17.676489: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-10-03 22:15:18.000318: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-10-03 22:15:18.000352: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5031/5031 [21:05<00:00,  3.97it/s]


In [8]:
len(results)

50304

In [9]:
import subprocess

In [10]:
with open('left', 'w') as fopen:
    fopen.write('\n'.join(list(results.keys())))
    
with open('right', 'w') as fopen:
    fopen.write('\n'.join(list(results.values())))

In [11]:
cmd = 'python3 align.py -s left -t right --priors align.priors --model 3 -f out.fwd -r out.rev --overwrite'
subprocess.run(cmd.split())

CompletedProcess(args=['python3', 'align.py', '-s', 'left', '-t', 'right', '--priors', 'align.priors', '--model', '3', '-f', 'out.fwd', '-r', 'out.rev', '--overwrite'], returncode=0)

In [12]:
!head -n 10 out.fwd

0-0 1-1 2-2 3-3 4-4
0-0 1-1 2-2 3-3 5-4 7-5 6-6
0-0 1-1
0-0 1-1 3-2 4-3 5-4 6-5 7-6 8-7 9-8
0-0 1-1 3-2 2-3 4-4 5-5 6-6 6-7
0-0 1-1 2-2 3-3 4-4 5-5 5-6 6-7 7-8 8-9 9-10 10-12 11-13 12-14 13-15 14-16 15-17 16-18 17-19 18-20 19-21 20-22
0-0 1-1 2-2 3-3 4-4
1-0 0-1 4-2 5-3 6-4
0-0 1-1 2-2 3-3
0-0 1-1 2-2 3-3 4-4 5-5 6-6 7-7 8-8 9-9 10-10 11-11


In [13]:
with open('out.fwd') as fopen:
    alignment = fopen.read().split('\n')[:-1]
len(alignment)

results_alignment = {}
for no, k in enumerate(results.keys()):
    results_alignment[k + results[k]] = alignment[no]

In [14]:
with open('out.rev') as fopen:
    alignment = fopen.read().split('\n')[:-1]
len(alignment)

results_alignment_rev = {}
for no, k in enumerate(results.keys()):
    results_alignment_rev[k + results[k]] = alignment[no]

In [15]:
data_ = copy.deepcopy(data)

In [16]:
answers_from_alignment = True

rs = []
for i in tqdm(range(len(data_))):
    try:
        for u in range(len(data_[i]['utterances'])):
            data_[i]['utterances'][u]['text_ms'] = results[data_[i]['utterances'][u]['text']]
            if 'segments' in data_[i]['utterances'][u]:
                for k in range(len(data_[i]['utterances'][u]['segments'])):
                    answer = {'text': data_[i]['utterances'][u]['segments'][k]['text'], 
                              'answer_start': data_[i]['utterances'][u]['segments'][k]['start_index']}
                    answer_translated, answer_translated_start = utils.extract_answer_translated(
                        answer,
                        results[data_[i]['utterances'][u]['segments'][k]['text']],
                        data_[i]['utterances'][u]['text'],
                        results[data_[i]['utterances'][u]['text']],
                        results_alignment[data_[i]['utterances'][u]['text'] + results[data_[i]['utterances'][u]['text']]],
                        answers_from_alignment,
                    )
                    data_[i]['utterances'][u]['segments'][k]['text_ms'] = answer_translated
                    data_[i]['utterances'][u]['segments'][k]['start_index_ms'] = answer_translated_start
                    data_[i]['utterances'][u]['segments'][k]['end_index_ms'] = answer_translated_start + len(answer_translated)
                    r = results[data_[i]['utterances'][u]['text']]
                    if r[answer_translated_start: answer_translated_start + len(answer_translated)] == results[data_[i]['utterances'][u]['segments'][k]['text']]:
                        data_[i]['utterances'][u]['segments'][k]['aligned'] = True
                    else:
                        data_[i]['utterances'][u]['segments'][k]['aligned'] = False
        rs.append(data_[i])
    except Exception as e:
        print(e)
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3056/3056 [00:03<00:00, 907.24it/s]


In [17]:
len(rs), len(data_)

(3056, 3056)

In [18]:
with open('movies-ms.json', 'w') as fopen:
    json.dump(rs, fopen)